In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.11.0  Device: cpu


In [3]:
df = pd.read_csv('C:/Users/JinnyeongHeo/Desktop/2022-1/추천시스템/HeartDisease/archive (1)/heart_2020_cleaned.csv')
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [4]:
print(df.shape)

(319795, 18)


In [5]:
label_encoder = preprocessing.LabelEncoder()

df['HeartDisease']= label_encoder.fit_transform(df['HeartDisease'])
df['Smoking']= label_encoder.fit_transform(df['Smoking'])
df['AlcoholDrinking']= label_encoder.fit_transform(df['AlcoholDrinking'])
df['Stroke']= label_encoder.fit_transform(df['Stroke'])
df['DiffWalking']= label_encoder.fit_transform(df['DiffWalking'])
df['Sex']= label_encoder.fit_transform(df['Sex'])
df['AgeCategory']= label_encoder.fit_transform(df['AgeCategory'])
df['Race']= label_encoder.fit_transform(df['Race'])
df['Diabetic']= label_encoder.fit_transform(df['Diabetic'])
df['PhysicalActivity']= label_encoder.fit_transform(df['PhysicalActivity'])
df['GenHealth']= label_encoder.fit_transform(df['GenHealth'])
df['Asthma']= label_encoder.fit_transform(df['Asthma'])
df['KidneyDisease']= label_encoder.fit_transform(df['KidneyDisease'])
df['SkinCancer']= label_encoder.fit_transform(df['SkinCancer'])

In [6]:
# X = pd.DataFrame(df, columns = df.columns)
X = df.drop(['HeartDisease'], axis=1)
y = pd.DataFrame(df, columns = ['HeartDisease'])
X.head()

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,16.60,1,0,0,3.0,30.0,0,0,7,5,2,1,4,5.0,1,0,1
1,20.34,0,0,1,0.0,0.0,0,0,12,5,0,1,4,7.0,0,0,0
2,26.58,1,0,0,20.0,30.0,0,1,9,5,2,1,1,8.0,1,0,0
3,24.21,0,0,0,0.0,0.0,0,0,11,5,0,0,2,6.0,0,0,1
4,23.71,0,0,0,28.0,0.0,1,0,4,5,0,1,4,8.0,0,0,0


In [7]:
X = X.to_numpy()
y = y.to_numpy()

In [8]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [9]:
# y['HeartDisease'].value_counts()

In [10]:
# torch_tensor = torch.tensor(df['targets'].values)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, shuffle = True, random_state=0)

In [12]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

torch.Size([239846, 17])
torch.Size([239846, 1])
torch.Size([79949, 17])
torch.Size([79949, 1])


In [13]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(17, 1) #self.linear = {w,b}
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [14]:
model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr=0.01)
    
nb_epochs = 100



for epoch in range(nb_epochs + 1):

    # H(x) 계산 => P(x=1)
    hypothesis = model(X_train) 

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 20 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Train Epoch {:4d}/{} Train Cost: {:.6f} Train Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Train Epoch    0/100 Train Cost: 0.387143 Train Accuracy 91.44%
Train Epoch   20/100 Train Cost: 0.303260 Train Accuracy 91.44%
Train Epoch   40/100 Train Cost: 0.289102 Train Accuracy 91.42%
Train Epoch   60/100 Train Cost: 0.285442 Train Accuracy 91.37%
Train Epoch   80/100 Train Cost: 0.282867 Train Accuracy 91.36%
Train Epoch  100/100 Train Cost: 0.280798 Train Accuracy 91.35%


In [15]:
model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr=0.01)
    
nb_epochs = 100




for epoch in range(nb_epochs + 1):

    # H(x) 계산 => P(x=1)
    hypothesis = model(X_test) 

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_test)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 20 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_test
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Test Epoch {:4d}/{} Test Cost: {:.6f} Test Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Test Epoch    0/100 Test Cost: 0.417486 Test Accuracy 86.94%
Test Epoch   20/100 Test Cost: 0.295136 Test Accuracy 91.37%
Test Epoch   40/100 Test Cost: 0.279959 Test Accuracy 91.34%
Test Epoch   60/100 Test Cost: 0.276951 Test Accuracy 91.30%
Test Epoch   80/100 Test Cost: 0.275557 Test Accuracy 91.27%
Test Epoch  100/100 Test Cost: 0.274495 Test Accuracy 91.26%


In [16]:
for name, param in model.named_parameters():
    parm = {}
    parm[name]=param.detach().numpy()  
    print(parm)

{'linear.weight': array([[-8.4085003e-02, -1.6101679e-01,  2.7959295e-02, -1.1532360e-01,
         3.7385266e-02,  2.9334500e-05,  2.1599615e-01, -1.8027842e-01,
         1.7252900e-01, -1.2564652e-01,  2.1269283e-01, -2.1238856e-01,
         1.0411509e-01, -1.1748338e-01, -3.4218378e-02,  8.4660783e-02,
        -2.1137697e-01]], dtype=float32)}
{'linear.bias': array([-0.06016985], dtype=float32)}


In [17]:
trained_w = np.array([-0.05709493, -0.1483185 , -0.20287965,  0.0196627 ,  0.04520231,
        -0.00625067,  0.20764819,  0.18666841,  0.15783934, -0.15874131,
        -0.04945571,  0.12815215, -0.07809287, -0.16721553, -0.02663539,
        -0.18812351,  0.15431833])
trained_b = np.array([-0.15068565])

In [18]:
def practice_exercise(inputs):
    x = 0
    for i in range(len(trained_w)):
        x += trained_w[i]*inputs[i]
    x += trained_b
    return x

In [19]:
test_sample_X = np.array(X_test[1160:1180])
test_sample_y = np.array(y_test[1160:1180])

print('outputs      label\n')

for i in range(len(test_sample_X)):
    x = []
    x.append(practice_exercise(test_sample_X[i]))
    x.append(test_sample_y[i])
    print(x[0],x[1])

outputs      label

[-1.83500796] [0.]
[-3.80138024] [0.]
[-2.44095826] [0.]
[-3.11195034] [0.]
[-1.6935326] [0.]
[-3.16170667] [0.]
[-1.11333294] [0.]
[-2.86128795] [0.]
[-3.32781749] [0.]
[-3.56311156] [0.]
[-2.2868853] [0.]
[-0.81954123] [0.]
[-2.19062338] [1.]
[-2.39412434] [0.]
[-1.44953226] [0.]
[0.05649307] [1.]
[-2.18116314] [1.]
[-2.80931583] [1.]
[-2.61233098] [0.]
[-2.69429381] [0.]
